<a href="https://colab.research.google.com/github/Bovi-analytics/SupplementalFiles-YongyanChen/blob/main/2_Data_editing_after_fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.co

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, DoubleType, FloatType
import pyspark
from pyspark.sql import functions as F,Window
from pyspark.sql.functions import year, col, array, monotonically_increasing_id, to_json, struct, round, datediff,unix_timestamp,udf,avg
import json
from urllib.request import  urlopen
import pyspark.sql.functions as F

# Load in fitting result per lactation

In [ ]:
FittingPerLactation= spark \
  .read \
  .parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/FittingPerLactation1112/')

In [ ]:
FittingPerLactation.show(3)

+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+
|HerdIdentifier|AnimalIdentifier|CalvingDate|Parity|DaysInMilk|Points|ScaleKg|Ramp|Offset|  Decay|Persistence|StdErrKg| M305Kg|
+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+
|           590|   NL 3233658025| 2008-07-31|   6.0|     377.0|   7.0|  52.76|34.8|   0.0|0.00233|      297.0|     1.9|10644.0|
|           590|   NL 3514206103| 2006-12-28|   3.0|     476.0|   8.0|  52.58| 8.5|   0.0|0.00219|      316.0|     1.8|11454.0|
|           590|   NL 4146636720| 2006-09-14|   1.0|     406.0|   6.0|  44.55|30.0|  -0.5|0.00184|      378.0|     1.9| 9759.0|
+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+
only showing top 3 rows



In [ ]:
FittingPerLactation.printSchema()

root
 |-- HerdIdentifier: integer (nullable = true)
 |-- AnimalIdentifier: string (nullable = true)
 |-- CalvingDate: date (nullable = true)
 |-- Parity: double (nullable = true)
 |-- DaysInMilk: double (nullable = true)
 |-- Points: double (nullable = true)
 |-- ScaleKg: double (nullable = true)
 |-- Ramp: double (nullable = true)
 |-- Offset: double (nullable = true)
 |-- Decay: double (nullable = true)
 |-- Persistence: double (nullable = true)
 |-- StdErrKg: double (nullable = true)
 |-- M305Kg: double (nullable = true)



In [ ]:
FittingPerLactation.select('ScaleKg','Ramp','Decay','Persistence','StdErrKg','M305Kg','Points').describe().show()

+-------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+------------------+
|summary|           ScaleKg|              Ramp|               Decay|      Persistence|            StdErrKg|              M305Kg|            Points|
+-------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+------------------+
|  count|          16676767|          16676767|            16676767|         16676767|            16674596|            16612565|          16676767|
|   mean| 43.42424388363418|24.385017305838023|0.002684000223172...|462.3720479515004|2.285239021253542E30|-3.07312246873255...| 7.895736026053491|
| stddev|11.196694672999707| 8.262488294588225|0.001115978197131...|69406.52921487276|4.938804847761733...|                 NaN|3.4434893629722203|
|    min|              1.26|              -0.5|                 0.0|             56.0|                 0.0|-3.44

In [ ]:
new_cols = ['ScaleKg','Ramp','Offset','Decay','Persistence']
for c in new_cols:
  FittingPerLactation.\
  withColumn('ParityGroup',F.when(F.col('Parity')==1,'1').
             otherwise('2+')).\
  groupBy('ParityGroup').\
  agg(F.mean(c),
    F.percentile_approx(c,0.5),
    F.stddev(c)).\
  show()

+-----------+-----------------+--------------------------------------+--------------------+
|ParityGroup|     avg(ScaleKg)|percentile_approx(ScaleKg, 0.5, 10000)|stddev_samp(ScaleKg)|
+-----------+-----------------+--------------------------------------+--------------------+
|         2+|47.17715545798857|                                 47.58|  10.552939396575196|
|          1|34.64810179118066|                                 34.17|   6.939123811812576|
+-----------+-----------------+--------------------------------------+--------------------+

+-----------+------------------+-----------------------------------+-----------------+
|ParityGroup|         avg(Ramp)|percentile_approx(Ramp, 0.5, 10000)|stddev_samp(Ramp)|
+-----------+------------------+-----------------------------------+-----------------+
|         2+| 22.15440454678542|                               22.5| 8.96056144463396|
|          1|29.601280502658522|                               29.8|1.143817551157831|
+-----------

In [ ]:
FittingPerLactation.approxQuantile(['ScaleKg','Ramp','Offset','Decay','Persistence','StdErrKg','M305Kg'], [0.5], 0.25)

[[42.66], [25.3], [-0.5], [0.00251], [278.0], [2.0], [8494.0]]

In [ ]:
FittingPerLactation.count()

16676767

In [ ]:
FittingPerLactation.filter((col('Decay')==0)).count()

62942

In [ ]:
FittingPerLactation.where(F.col("M305Kg").isNull()).count()

64202

In [ ]:
FittingPerLactation.where(F.col("Persistence").isNull()).count()

0

# Recalculate decay and M305

In [ ]:
import math
from pyspark.sql.functions import udf
M305_udf=udf(lambda Scale,Ramp,Offset,Decay:(Scale-Scale*math.exp(-305*Decay))/Decay+(Scale*Ramp*math.exp(Offset/Ramp)*(-1+math.exp(-305*((1/Ramp)+Decay))))/(2+2*Ramp*Decay),DoubleType())

In [ ]:
FittingPerLactation=FittingPerLactation.\
  withColumn('Decay',F.when(FittingPerLactation.Decay==0,0.693/FittingPerLactation.Persistence).otherwise(FittingPerLactation.Decay))

In [ ]:
FittingPerLactation=FittingPerLactation.filter('Ramp>0')

In [ ]:
FittingPerLactation=FittingPerLactation.\
  withColumn('M305Kg',F.when(FittingPerLactation_1.M305Kg.isNull(),M305_udf(FittingPerLactation_1.ScaleKg,FittingPerLactation_1.Ramp,FittingPerLactation_1.Offset,FittingPerLactation_1.Decay)).otherwise(FittingPerLactation_1.M305Kg))

In [ ]:
FittingPerLactation.select('ScaleKg','Ramp','Decay','Persistence','StdErrKg','M305Kg','Points').describe().show()

+-------+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+
|summary|           ScaleKg|              Ramp|               Decay|       Persistence|          StdErrKg|            M305Kg|            Points|
+-------+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+
|  count|          16674591|          16674591|            16674591|          16674591|          16674591|          16674591|          16674591|
|   mean|  43.4236263245402|24.388201089935478|0.002686531855513955|462.40053576126695| 2.035951970271083| 8502.416728651928| 7.895874027734774|
| stddev|11.197277429372612| 8.258325227197549|0.001110592681658...| 69411.05748970865|1.1541129752016521|1892.6148422999186|3.4433423814670068|
|    min|              1.26|               0.1| 7.30348911282157E-9|              56.0|               0.0|             277.0|     

In [ ]:
FittingPerLactation=FittingPerLactation.withColumn('ParityGroup',F.when(F.col('Parity')==1,'1').otherwise('2+'))

In [ ]:
FittingPerLactation.agg(F.countDistinct("HerdIdentifier"),F.countDistinct("AnimalIdentifier")).collect()

[Row(count(HerdIdentifier)=17580, count(AnimalIdentifier)=5894461)]

#Excluded extreme lactation curve characteristics

In [ ]:
FittingPerLactation.approxQuantile(['ScaleKg','Ramp','Persistence'], [0.01,0.05,0.95,0.99], 0.00001)

[[20.34, 26.32, 62.6, 71.68],
 [3.9, 8.2, 36.7, 46.2],
 [119.0, 150.0, 664.0, 1305.0]]

In [ ]:
QuantilesScaleKg = FittingPerLactation.agg(F.expr('percentile(ScaleKg,array(0.01,0.99))').alias('Q1')).select('Q1').collect()[0][0]
QuantilesRamp = FittingPerLactation.agg(F.expr('percentile(Ramp,array(0.01,0.99))').alias('Q1')).select('Q1').collect()[0][0]
QuantilesPersistence = FittingPerLactation.agg(F.expr('percentile(Persistence,array(0.01,0.99))').alias('Q1')).select('Q1').collect()[0][0]

In [ ]:
FittingPerLactation.\
  select('ScaleKg','Ramp','Persistence').\
  summary().\
  show()

+-------+------------------+-----------------+------------------+
|summary|           ScaleKg|             Ramp|       Persistence|
+-------+------------------+-----------------+------------------+
|  count|          16674591|         16674591|          16674591|
|   mean|    43.42362632454|24.38820108993547|462.40053576126695|
| stddev|11.197277429372397|8.258325227197652| 69411.05748970821|
|    min|              1.26|              0.1|              56.0|
|    25%|             35.27|             21.7|             209.0|
|    50%|             42.98|             23.3|             266.0|
|    75%|              50.4|             29.8|             364.0|
|    max|            136.48|            103.0|       9.4886155E7|
+-------+------------------+-----------------+------------------+



In [ ]:
FittingPerLactationFiltered = FittingPerLactation. \
  filter((col('ScaleKg')>QuantilesScaleKg[0]) &\
      (col('ScaleKg')<QuantilesScaleKg[1]) &\
      (col('Ramp')>QuantilesRamp[0]) &\
      (col('Ramp')<QuantilesRamp[1]) &\
      (col('Persistence')>QuantilesPersistence[0]) &\
      (col('Persistence')<QuantilesPersistence[1])
  )

In [ ]:
FittingPerLactationFiltered.\
  select('ScaleKg','Ramp','Persistence').\
  summary().\
  show()

+-------+-----------------+------------------+------------------+
|summary|          ScaleKg|              Ramp|       Persistence|
+-------+-----------------+------------------+------------------+
|  count|         15726779|          15726779|          15726779|
|   mean|43.36264926520977|24.276854141615818|310.00849442851586|
| stddev| 10.3067715165052| 7.549765210298293| 157.8070939571753|
|    min|            20.35|               4.0|             120.0|
|    25%|            35.56|              21.8|             211.0|
|    50%|            43.02|              23.2|             266.0|
|    75%|            50.18|              29.8|             360.0|
|    max|            71.66|              46.1|            1304.0|
+-------+-----------------+------------------+------------------+



In [ ]:
FittingPerLactationFiltered.show(2)

+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+
|HerdIdentifier|AnimalIdentifier|CalvingDate|Parity|DaysInMilk|Points|ScaleKg|Ramp|Offset|  Decay|Persistence|StdErrKg| M305Kg|ParityGroup|
+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+
|           590|   NL 3233658025| 2008-07-31|   6.0|     377.0|   7.0|  52.76|34.8|   0.0|0.00233|      297.0|     1.9|10644.0|         2+|
|           590|   NL 3514206103| 2006-12-28|   3.0|     476.0|   8.0|  52.58| 8.5|   0.0|0.00219|      316.0|     1.8|11454.0|         2+|
+--------------+----------------+-----------+------+----------+------+-------+----+------+-------+-----------+--------+-------+-----------+
only showing top 2 rows



In [ ]:
FittingPerLactationFiltered.agg(F.countDistinct("HerdIdentifier"),F.countDistinct("AnimalIdentifier")).collect()

[Row(count(HerdIdentifier)=17579, count(AnimalIdentifier)=5773825)]

In [ ]:
FittingPerLactationFiltered.\
  repartition(1).\
  write.\
  parquet(path='/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/FittingPerLactationFiltered',mode="overwrite")

In [ ]:
FittingPerLactationFiltered = spark \
  .read \
  .parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/FittingPerLactationFiltered')